# Explaining notes:

- Mesh is imported then the kernel function is defined on the mesh using the package. 
- Then regular discretised BO is done on the mesh's nodes. 
- You can change all the parameters of:
   - The Bayesian optimisation loop: num_iterations etc.
   - The acquisition function (it's EI): exploration, exploitation etc..
   - The mesh kernel. 
- The plotly thing shows traces which can be shown and hidden by clicking on them. For this case each trace is one of the "functions" (i.e nx1 vectors where n is the number of nodes in the mesh) applied to the mesh.
- You can switch between the "chemistry" function and the dummy function when doing the torus but I don't think there's anything interesting. 
- the torus mesh is a bit weird and so the algorithm never explores the inside of the torus, but it does fine for the teddy bear mesh




Contents of this ipynb:
- Doing BO on the mesh. Showing with two different BO loops (and then plotting the posterior mu_1, mu_2 functions respectively). You can tweak the number of iterations that each do 
- Visualisation of the kernel spreading covariance throughout the shape. 

## Importing everything

In [4]:
!pip install matplotlib ipympl geometric_kernels kaleido plotly scipy rdkit 


import numpy as np
import random
from scipy.stats import norm
import geometric_kernels
from geometric_kernels.spaces import Mesh
from geometric_kernels.kernels import MaternGeometricKernel, MaternKarhunenLoeveKernel
import matplotlib as mpl
import matplotlib.pyplot as plt
import kaleido
import plotly
import plotly.io as pio
pio.renderers.default = "browser" # "browser" opens in browser. "vscode" in vscode (for the notebook version; doesn't work with py) 
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from Plotting import *
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, rdMolTransforms, Descriptors
from rdkit.Chem import rdMolTransforms
# import matplotlib 
# import matplotlib.pyplot as plt
# # !pip install ipympl
# import ipympl
# %matplotlib notebook
from rdkit import Chem
from rdkit.Chem import rdMolTransforms

INFO (geometric_kernels): Numpy backend is enabled. To enable other backends, don't forget to `import geometric_kernels.*backend name*`.
INFO (geometric_kernels): We may be suppressing some logging of external libraries. To override the logging policy, call `logging.basicConfig`.
c:\Users\aum39\OneDrive\Desktop\Research project current SOP\Sphere example for Hybrid Bayesian optimisation\.venv\Lib\site-packages\spherical_harmonics\fundamental_set.py:21: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
INFO (rdkit): Enabling RDKit 2025.09.1 jupyter extensions


## Chemistry-specific stuff

In [5]:
def set_double_torsion_and_get_energy(molecule, angle1, angle2):
    """
    Set two torsion angles and return minimized energy.
    """
    mol = Chem.Mol(molecule)
    
    # Find rotatable bonds
    rotatable_bonds = mol.GetSubstructMatches(
        Chem.MolFromSmarts('[!$(*#*)&!D1]-!@[!$(*#*)&!D1]')
    )
    
    if len(rotatable_bonds) < 2:
        return None
    
    conform = mol.GetConformer()
    angles = [angle1, angle2]
    
    # Set both torsion angles directly first
    for idx, bond in enumerate(rotatable_bonds[:2]):
        i, j = bond
        i_neighbours = [n.GetIdx() for n in mol.GetAtomWithIdx(i).GetNeighbors() 
                       if n.GetIdx() != j][0]
        j_neighbours = [n.GetIdx() for n in mol.GetAtomWithIdx(j).GetNeighbors() 
                       if n.GetIdx() != i][0]
        rdMolTransforms.SetDihedralDeg(conform, i_neighbours, i, j, j_neighbours, 
                                       angles[idx])
    
    # Now create force field and minimize with constraints
    props = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant='MMFF94')
    ff = AllChem.MMFFGetMoleculeForceField(mol, props)
    
    # Add torsion constraints for both bonds
    for idx, bond in enumerate(rotatable_bonds[:2]):
        i, j = bond
        i_neighbours = [n.GetIdx() for n in mol.GetAtomWithIdx(i).GetNeighbors() 
                       if n.GetIdx() != j][0]
        j_neighbours = [n.GetIdx() for n in mol.GetAtomWithIdx(j).GetNeighbors() 
                       if n.GetIdx() != i][0]
        ff.MMFFAddTorsionConstraint(i_neighbours, i, j, j_neighbours, 
                                    False, angles[idx] - 0.5, angles[idx] + 0.5, 
                                    1000.0)
    
    ff.Minimize()
    return ff.CalcEnergy()


pentane = Chem.MolFromSmiles('CCCCC')
AllChem.EmbedMolecule(pentane)



[10:50:40] Molecule does not have explicit Hs. Consider calling AddHs()


0

## Mesh and kernel set up

In [6]:
MeshFolder_dir = Path.cwd() #/ "My meshes"
print( "DIR:", MeshFolder_dir)
Filename = "100 POINTS ONE.obj"
mesh = Mesh.load_mesh(str(MeshFolder_dir / Filename))
print(mesh.num_vertices)


#Kernel stuff:
kernel = MaternGeometricKernel(mesh)

LENGTH_SCALE,NU = 2.86, 0.5
VARIANCE = "NEEDS_CONFIGURING"  #variance isn't part of the package but it can be "easily implemented" by multiplying the kernel by a constant and then optimising for it

params = kernel.init_params()
params["lengthscale"] = np.array([LENGTH_SCALE])
params["nu"] = np.array([NU])



DIR: c:\Users\aum39\OneDrive\Desktop\Research project current SOP\Sphere example for Hybrid Bayesian optimisation\Main
3990


## Objective functions (choose one)


In [7]:
def actual_chemical_f(x): #return this immediately TODO: reutnr 
    #x is a node index.
    if isinstance(x, np.ndarray):
        x = x.flatten()

    rho, theta = divmod(x, 360)
    rho, theta = int(rho), int(theta)
    # rho, theta = theta, rho #TODO: delete immediately
    return set_double_torsion_and_get_energy(pentane, rho, theta)

def f(x):
    #assuming that x is a node index.
    if isinstance(x, np.ndarray): euclidean_coord =  mesh.vertices[np.int64(x.flatten()) -1].flatten()
    else:
        euclidean_coord = mesh.vertices[x-1]
    return euclidean_coord[0]
    # return np.array(3)
j = np.vectorize(f)

## BO setup

In [8]:
def expected_improvement(mu, sigma, f_best, xi=0.0):
    """
    Calculates the Expected Improvement (EI) for a set of points.

    Args:
        mu (np.ndarray): The posterior mean vector.
        sigma (np.ndarray): The posterior standard deviation vector.
        f_best (float): The best observed function value.
        xi (float): The exploration-exploitation trade-off parameter.

    Returns:
        np.ndarray: The EI vector.
    """
    # Calculate the standardized improvement (Z)
    xi = 0.9
    with np.errstate(divide='ignore'):  # suppress divide by zero warning
        
        mu = mu.reshape(-1,1) # TODO: delete this if it causes issues. 
        sigma = sigma.reshape(-1,1)
        Z = (f_best - mu - xi) / sigma #TODO: this and the one below may be the wrong expression. 
        
    # Calculate the EI for non-zero sigma
    ei = (f_best - mu - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    
    # Handle the case where sigma is zero
    ei = np.where(sigma > 1e-10, ei, 0.0)
    
    return ei


def BO_loop_fixed(num_iterations, x_obs=None, objective_func=None, 
                  exploration_weight=0.1):
    """
    Fixed BO loop with better exploration of inner torus surface.
    
    Args:
        exploration_weight: Add random exploration with this probability
    """
    if objective_func is None:
        objective_func = f
    if x_obs is None:
        x_obs = x_observed

    num_verts = mesh.num_vertices
    whole_domain = np.atleast_2d(np.arange(1, num_verts + 1)).T

    y_observed = np.atleast_2d(np.apply_along_axis(objective_func, 1, x_obs)).reshape(-1, 1)
    K_XX_prior = kernel.K(params, whole_domain - 1, whole_domain - 1)
    mu_prior_vector = np.zeros((num_verts, 1))

    exploration_points = []

    for i in range(num_iterations):
        # GP Model Fitting
        m_vector = mu_prior_vector[x_obs.flatten() - 1]
        K_xX = kernel.K(params, x_obs - 1, whole_domain - 1)
        K_xx = kernel.K(params, x_obs - 1, x_obs - 1)
        K_Xx = K_xX.T
        
        K_xx_stable = K_xx + np.eye(K_xx.shape[0]) * 1e-6
        C_inv = np.linalg.pinv(K_xx_stable)

        mew_vec = mu_prior_vector + K_Xx @ C_inv @ (y_observed - m_vector)
        Current_K_matrix = K_XX_prior - K_Xx @ C_inv @ K_xX
        Sigma_vec = np.diag(Current_K_matrix).copy().reshape(-1, 1)
        Sigma_vec[Sigma_vec < 0] = 0 

        # Acquisition Function
        EI_vec = expected_improvement(mew_vec, np.sqrt(Sigma_vec), np.min(y_observed))
        
        
        next_point = np.argmax(EI_vec) + 1
        
        next_point = np.atleast_2d(next_point)

        # Update
        y_next = np.atleast_2d(objective_func(next_point - 1))
        x_obs = np.vstack((x_obs, next_point))
        y_observed = np.vstack((y_observed, y_next))

    
    return mew_vec, Sigma_vec, EI_vec, x_obs, y_observed

## BO 

In [13]:
objective_vals = j(np.arange(0, mesh.num_vertices, 1))
pio.renderers.default = "browser"

objective_plot_kwargs = dict(
    name = "objective value",
    marker = dict(
        size = 10,
        colorscale = 'hot'
    )
)
objective_trace = vector_values_to_mesh_trace(mesh, objective_vals, **objective_plot_kwargs)
hover_kwargs = dict(
    customdata = np.hstack([np.atleast_2d(objective_vals).T, np.atleast_2d(np.arange(1, mesh.num_vertices + 1)).T]),
    hovertemplate = ("x : %{x:.2f}<br>" + "y : %{y:.2f}<br>" + "z : %{z:.2f}<br>" + "objective value: %{customdata[0]:.2f}<br>" + "node index: %{customdata[1]}"),
)
objective_trace = add_custom_hover_data(objective_trace, **hover_kwargs)



# plot 2 different results (posterior mean, posterior stdev, x observed, y observed) with 2 different num_iterations


initial_point = np.array ([[597]])
mu_1, sigma_1, ei_1, X_1, Y_1 = BO_loop_fixed(1, x_obs = initial_point, objective_func = f)
mu_2, sigma_2, ei_2, X_2, Y_2 = BO_loop_fixed(5,x_obs = initial_point,  objective_func = f)
mu_3, sigma_3, ei_3, X_3, Y_3 = BO_loop_fixed(10,x_obs = initial_point,  objective_func = f)

advanced_fig = go.Figure()

mu_1_trace = vector_values_to_mesh_trace(mesh, mu_1)
mu_2_trace = vector_values_to_mesh_trace(mesh, mu_2)
mu_3_trace = vector_values_to_mesh_trace(mesh, mu_3)
mu_2_trace = add_custom_hover_data(mu_2_trace, customdata = mu_2)
mu_1_trace = add_custom_hover_data(mu_1_trace, customdata = mu_1)
mu_3_trace = add_custom_hover_data(mu_3_trace, customdata = mu_3)

advanced_fig.add_trace(mu_1_trace)
advanced_fig.add_trace(mu_2_trace)
advanced_fig.add_trace(mu_3_trace)
advanced_fig.add_trace(objective_trace)




####plotting the points explored by the algoritjm

euclidean_coord_of_point_plotted = mesh.vertices[np.int64(X_2.flatten()-1)]
third_euclidean_coord_of_point_plotted = mesh.vertices[np.int64(X_3.flatten()-1)]
point_plot_trace = go.Scatter3d(
    x = np.array(euclidean_coord_of_point_plotted[:,0]),
    y = np.array(euclidean_coord_of_point_plotted[:,1]),
    z = np.array(euclidean_coord_of_point_plotted[:,2]),
    marker = dict(
        color = "cyan"
    ),
    mode = "markers"
)

second_point_plot_trace = go.Scatter3d(
    x = np.array(third_euclidean_coord_of_point_plotted[:,0]),
    y = np.array(third_euclidean_coord_of_point_plotted[:,1]),
    z = np.array(third_euclidean_coord_of_point_plotted[:,2]),
    marker = dict(
        color = "purple"
    ),
    mode = "markers"
)

advanced_fig.add_trace(point_plot_trace)
advanced_fig.add_trace(second_point_plot_trace)



fig_show(advanced_fig) #TODO: add this back

print("the sampled values are (sorted ascending)", sorted(list(Y_2)), ".While the true best is", np.min(objective_vals), "which is at index:", np.argmax(-1*objective_vals))
print("the initial point's value is", objective_vals[initial_point.squeeze()])







the sampled values are (sorted ascending) [array([-0.95602989]), array([-0.94631714]), array([-0.92845231]), array([-0.67470318]), array([-0.33518577]), array([0.81917357])] .While the true best is -1.05024886 which is at index: 449
the initial point's value is 0.81917357


# Manipulating the plot:
In each visualisation, there are "traces" which are basically like "axes" in numpy. You can click each to show/hide them. The objective one is the objective function visualised on the mesh and the trace 1 and trace 2 are the posterior means predicted by the GP's

## Plot to show kernel geometry-awareness

In [12]:
objective_vals = j(np.arange(0, mesh.num_vertices, 1))
pio.renderers.default = "browser"

objective_plot_kwargs = dict(
    name = "objective value",
    marker = dict(
        size = 10,
        colorscale = 'hot'
    )
)
objective_trace = vector_values_to_mesh_trace(mesh, objective_vals, **objective_plot_kwargs)
hover_kwargs = dict(
    customdata = np.hstack([np.atleast_2d(objective_vals).T, np.atleast_2d(np.arange(1, mesh.num_vertices + 1)).T]),
    hovertemplate = ("x : %{x:.2f}<br>" + "y : %{y:.2f}<br>" + "z : %{z:.2f}<br>" + "objective value: %{customdata[0]:.2f}<br>" + "node index: %{customdata[1]}"),
)
objective_trace = add_custom_hover_data(objective_trace, **hover_kwargs)



# plot 2 different results (posterior mean, posterior stdev, x observed, y observed) with 2 different num_iterations


initial_point = np.array ([[np.random.randint(1, mesh.num_vertices)]])
mu_1, sigma_1, ei_1, X_1, Y_1 = BO_loop_fixed(1, x_obs = initial_point, objective_func = f)
mu_2, sigma_2, ei_2, X_2, Y_2 = BO_loop_fixed(5,x_obs = initial_point,  objective_func = f)
mu_3, sigma_3, ei_3, X_3, Y_3 = BO_loop_fixed(10,x_obs = initial_point,  objective_func = f)

advanced_fig = go.Figure()

mu_1_trace = vector_values_to_mesh_trace(mesh, mu_1)
mu_2_trace = vector_values_to_mesh_trace(mesh, mu_2)
mu_3_trace = vector_values_to_mesh_trace(mesh, mu_3)
mu_2_trace = add_custom_hover_data(mu_2_trace, customdata = mu_2)
mu_1_trace = add_custom_hover_data(mu_1_trace, customdata = mu_1)
mu_3_trace = add_custom_hover_data(mu_3_trace, customdata = mu_3)

advanced_fig.add_trace(mu_1_trace)
advanced_fig.add_trace(mu_2_trace)
advanced_fig.add_trace(mu_3_trace)
advanced_fig.add_trace(objective_trace)




####plotting the points explored by the algoritjm

euclidean_coord_of_point_plotted = mesh.vertices[np.int64(X_2.flatten()-1)]
third_euclidean_coord_of_point_plotted = mesh.vertices[np.int64(X_3.flatten()-1)]
point_plot_trace = go.Scatter3d(
    x = np.array(euclidean_coord_of_point_plotted[:,0]),
    y = np.array(euclidean_coord_of_point_plotted[:,1]),
    z = np.array(euclidean_coord_of_point_plotted[:,2]),
    marker = dict(
        color = "cyan"
    ),
    mode = "markers"
)

second_point_plot_trace = go.Scatter3d(
    x = np.array(third_euclidean_coord_of_point_plotted[:,0]),
    y = np.array(third_euclidean_coord_of_point_plotted[:,1]),
    z = np.array(third_euclidean_coord_of_point_plotted[:,2]),
    marker = dict(
        color = "purple"
    ),
    mode = "markers"
)

advanced_fig.add_trace(point_plot_trace)
advanced_fig.add_trace(second_point_plot_trace)



fig_show(advanced_fig) #TODO: add this back

print("the sampled values are (sorted ascending)", sorted(list(Y_2)), ".While the true best is", np.min(objective_vals), "which is at index:", np.argmax(-1*objective_vals))
print("the initial point's value is", objective_vals[initial_point.squeeze()])




the sampled values are (sorted ascending) [array([-0.99132395]), array([-0.94631714]), array([-0.67470318]), array([-0.33518577]), array([-0.06357181]), array([0.45132577])] .While the true best is -1.05024886 which is at index: 449
the initial point's value is 0.45132577


In [17]:
print(max([np.linalg.norm(x) for x in mesh.vertices]))

1.0502859424392292
